In [1]:
import os 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,optimizers,datasets

C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", n

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # or any {'0','1','2'}

In [3]:
(x,y),_ = datasets.mnist.load_data()

In [4]:
x.shape

(60000, 28, 28)

In [5]:
y.shape

(60000,)

In [6]:
def prepare_mnist_features_and_labels(x,y):
    x = tf.cast(x,tf.float32) / 255.0
    y = tf.cast(y,tf.int64)
    return x,y

In [7]:
def mnist_dataset():
    (x,y),_ = datasets.mnist.load_data()
    ds = tf.data.Dataset.from_tensor_slices((x,y))
    ds = ds.map(prepare_mnist_features_and_labels)
    ds = ds.take(20000).shuffle(20000).batch(100)
    return ds

In [8]:
temp = mnist_dataset()

In [9]:
features,label = iter(temp).next()
print("example features:", features[0])
print("example label:", label[0])

example features: tf.Tensor(
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.  

In [10]:
model = keras.Sequential([
    layers.Reshape(target_shape=(28*28,),input_shape=(28,28)),
    layers.Dense(100,activation='relu'),
    layers.Dense(100,activation='relu'),
    layers.Dense(10)
])

In [11]:
optimizer = optimizers.Adam()

In [17]:
@tf.function
def compute_loss(logits,labels):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=labels))
@tf.function
def compute_accuracy(logits,labels):
    predictions = tf.argmax(logits,axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions,labels),tf.float32))
@tf.function
def train_one_step(model,optimizer,x,y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits,y)
    # 计算梯度
    grads = tape.gradient(loss,model.trainable_variables)
    # 更新权重
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    accuracy = compute_accuracy(logits,y)
    return loss,accuracy
def train(epoch,model,optimizer):
    train_ds = mnist_dataset()
    loss = 0.0
    accuracy = 0.0
    for step,(x,y) in enumerate(train_ds):
        loss,accuracy = train_one_step(model,optimizer,x,y)
#         print(step)
        if step % 500 == 0:
            print('epoch',epoch,':loss',loss.numpy(),':accuracy',accuracy.numpy())
    return loss,accuracy

In [18]:
for epoch in range(20):
    loss,accuracy = train(epoch,model,optimizer)
print('Final epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 :loss 1.4024983e-05 :accuracy 1.0
epoch 1 :loss 2.84906e-06 :accuracy 1.0
epoch 2 :loss 6.690933e-06 :accuracy 1.0
epoch 3 :loss 3.1565908e-06 :accuracy 1.0
epoch 4 :loss 5.4226257e-06 :accuracy 1.0
epoch 5 :loss 0.020009425 :accuracy 0.99
epoch 6 :loss 0.013557776 :accuracy 0.99
epoch 7 :loss 0.000598106 :accuracy 1.0
epoch 8 :loss 0.00044714508 :accuracy 1.0
epoch 9 :loss 0.000171785 :accuracy 1.0
epoch 10 :loss 6.921814e-05 :accuracy 1.0
epoch 11 :loss 9.915712e-05 :accuracy 1.0
epoch 12 :loss 3.449712e-05 :accuracy 1.0
epoch 13 :loss 2.1767373e-05 :accuracy 1.0
epoch 14 :loss 2.2151771e-05 :accuracy 1.0
epoch 15 :loss 9.6922515e-05 :accuracy 1.0
epoch 16 :loss 7.321668e-05 :accuracy 1.0
epoch 17 :loss 2.5825942e-05 :accuracy 1.0
epoch 18 :loss 4.9711354e-05 :accuracy 1.0
epoch 19 :loss 6.7782334e-05 :accuracy 1.0
Final epoch 19 : loss 3.5051577e-05 ; accuracy 1.0
